In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pandas as pd
import numpy as np
import string
from sklearn.utils import shuffle

# nltk downloads
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Load data from csv files, change to json later

K_path = "TFIDF-K.csv"
K_df = pd.read_csv(K_path)
Y_path = "TFIDF-Y.csv"
Y_df = pd.read_csv(Y_path)
H_path = "TFIDF-H.csv"
H_df = pd.read_csv(H_path)
N_path = "TFIDF-N.csv" 
N_df = pd.read_csv(N_path)
Z_path = "TFIDF-Z.csv"
Z_df = pd.read_csv(Z_path)

Stemming and stopword removal functions


In [ ]:
stop = set(stopwords.words('english') + list(string.punctuation))
porter = PorterStemmer()

def remove_stopwords(sentence):
  tokens = [i for i in word_tokenize(sentence.lower()) if i not in stop]
  filtered = ' '.join([str(w) for w in tokens])
  return filtered

def stem(sentence):
    tokens = word_tokenize(sentence)
    stemmed_tokens = [porter.stem(w) for w in tokens]
    stemmed = ' '.join(stemmed_tokens)
    return stemmed

Choose correct option based on highest word overlap between query and description

In [ ]:
# Helper function to find number of overlapping words between two sentences
def calc_overlap(s1, s2):
  s1_list = s1.split(' ')
  s2_list = s2.split(' ')
  return len(list(set(s1_list)&set(s2_list)))

# General function to determine correct answer based on word overlap after pre-processing
def word_overlap(df, name):
  results = []
  correct = 0
  total = len(df)
  for i in range(len(df)):
    try:
      options = [df.loc[i, "Option 1"], df.loc[i, "Option 2"], df.loc[i, "Option 3"], df.loc[i, "Option 4"], df.loc[i, "Option 5"]]
      query = df.loc[i, "Query"]
      answer = df.loc[i, "Correct Answer"]
    except:
      continue
    
    options_str = [str(i) for i in options]
    cleaned_query = stem(remove_stopwords(str(query)))
    
    overlap = []
    for option in options_str:
      cleaned_option = stem(remove_stopwords(option))
      num_overlap = calc_overlap(cleaned_query, cleaned_option)
      overlap.append(num_overlap)

    overlap, options = shuffle(overlap, options, random_state=0)
    # Ties in argmax are broken by picking first element
    ind = np.argmax(overlap) 
    result = 0
    if (options[ind]) == answer:
      correct += 1
      result = 1
    results.append([result, "Query: " + str(df.loc[i, "Query"]), "Recommended: " + str(options[ind])])

  print("Results for {}:".format(name))
  print("Total correct answers: {} out of {}".format(correct, total))
  print(results)

In [ ]:
word_overlap(K_df, "K")
word_overlap(Y_df, "Y")
word_overlap(H_df, "H")
word_overlap(N_df, "N")
word_overlap(Z_df, "Z")

Results for K:
Total correct answers: 22 out of 100
[[1, 'Query: Can I have an Egyptian dessert?', 'Recommended: An Egyptian fruit cap made of dates and fruits'], [0, 'Query: I want a South African vegetarian cuisine', 'Recommended: Ribs made with south african spices'], [0, "Query: I'm a Kenyan long distance runner and I want a main dish full of protein", 'Recommended: Kenyan collard greens simmered with tomatoes'], [1, 'Query: I want to try a soup from West Africa', 'Recommended: West Africa soup made of squash and flavoured with chili powder and smoothed out with coconut milk'], [0, 'Query: I want to make Quebec poutine', 'Recommended: Sweet Quebec beans made with maple syrup and molasses'], [0, "Query: I'm a vegan who hates tofu but wants to eat burgers", 'Recommended: Burger made from only spinach and firm tofu'], [0, "Query: I want a Canadian meat dish that's filling", 'Recommended: Filling beef and potatoes'], [1, 'Query: I bought some fancy caviar to make a dip', 'Recommended: 